In [1]:
#all imports
import carla #the sim library itself
import random #to pick random spawn point
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import time
from keras.models import load_model
import pandas as pd
import sys
sys.path.append("D:\WindowsNoEditor\PythonAPI\carla")  # Adjust this path
import carla
import pygame
from agents.navigation.global_route_planner import GlobalRoutePlanner
from agents.navigation.behavior_agent import BehaviorAgent


pygame 2.6.1 (SDL 2.28.4, Python 3.7.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# connect to the sim 
client = carla.Client('localhost', 2000)
# optional to load different towns
client.set_timeout(15)
client.load_world('Town06')
#define environment/world and get possible places to spawn a car
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()
#look for a blueprint of Mini car
vehicle_bp = world.get_blueprint_library().filter('nissan')
#spawn a car in a random location

start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

# move simulator view to the car
spectator = world.get_spectator()
start_point.location.z = start_point.location.z+1 #start_point was used to spawn the car but we move 1m up to avoid being on the floor
spectator.set_transform(start_point)


In [6]:
# Get the map object
carla_map = world.get_map()
all_corners = list()

# Get the waypoint for the starting position (assumes the car spawns in the leftmost lane)
start_waypoint = carla_map.get_waypoint(
    start_point.location, 
    project_to_road=True, 
    lane_type=carla.LaneType.Driving
)

# Shift 4 lanes to the right
target_waypoint = start_waypoint
for _ in range(4):
    if target_waypoint.get_right_lane():
        target_waypoint = target_waypoint.get_right_lane()
    else:
        print("The road does not have 4 lanes to the right!")
        target_waypoint = None
        break

# Ensure the target lane exists before proceeding
if target_waypoint:
    # Define the starting location in the shifted lane
    start_location = target_waypoint.transform.location

    # Define the size of the "plane" (length, width)
    length = 562.4  # Length of the "plane" (this determines how far ahead the lines go)
    width = 3.5   # Width of the "plane" (lane width)

    # Number of lines to draw, to cover most of the center of the lane
    num_lines = 200  # You can increase this number for more lines

    # Define the spacing between lines
    line_spacing = width / num_lines

    # Draw multiple lines in the center of the lane to simulate a "plane"
    for i in range(num_lines):
        # Calculate the y-coordinate for each line to simulate a series of parallel lines
        y_offset = (i - num_lines / 2) * line_spacing  # Center the lines in the lane

        # Define the start and end location of each line
        start_line_location = carla.Location(
            x=start_location.x, 
            y=start_location.y + y_offset, 
            z=start_location.z
        )
        
        # End point 25 meters ahead
        end_line_location = carla.Location(
            x=start_line_location.x - length, 
            y=start_line_location.y, 
            z=start_line_location.z
        )

        # Define a color (e.g., Yellow)
        color = carla.Color(r=0, g=1, b=0, a=0)

        # Draw the line on the road
        world.debug.draw_line(
            start_line_location, 
            end_line_location, 
            thickness=0.05, 
            color=color, 
            life_time=0
        )
else:
    print("Could not draw the lines. Target lane does not exist!")


# Ensure the target lane exists before proceeding
if target_waypoint:
    start_location = target_waypoint.transform.location


     # Define coil size
    coil_width = 6 # Width of each coil (adjust as needed)
    coil_height = 1  # Height (adjust as needed)

    # Define coil positions based on the image (5m and 8m spacing, total 39m)
    coil_positions = [ i for i in range(5,int(length),1+coil_width)]
      
   

    
    coil_color1 = carla.Color(r=1, g=1, b=1)
    coil_color2 = carla.Color(r=1, g=1, b=0)
    coil_color = coil_color2

    # Draw each coil as a rectangle
    for i,pos in enumerate(coil_positions):

        if (i+1)%3==0 and coil_color == coil_color2:
            coil_color = coil_color1
        elif (i+1)%3==0 and coil_color == coil_color1:
            coil_color = coil_color2


        # Define coil color (Yellow)
        
        coil_center_x = start_location.x - pos  # Adjust X position
        coil_center_y = start_location.y  # Keep Y position in lane center

        # Define 4 corners of the rectangle
        corners = [
            carla.Location(x=coil_center_x - coil_width/2, y=coil_center_y - coil_height/2, z=start_location.z),
            carla.Location(x=coil_center_x + coil_width/2, y=coil_center_y - coil_height/2, z=start_location.z),
            carla.Location(x=coil_center_x + coil_width/2, y=coil_center_y + coil_height/2, z=start_location.z),
            carla.Location(x=coil_center_x - coil_width/2, y=coil_center_y + coil_height/2, z=start_location.z)
        ]
        all_corners.append(corners)

        # Draw the rectangle by connecting its corners
        for i in range(4):
            world.debug.draw_line(
                corners[i], 
                corners[(i+1) % 4], 
                thickness=0.1, 
                color=coil_color, 
                life_time=0
            )
else:
    print("Could not draw the coils. Target lane does not exist!")


In [ ]:
class BatteryManager:
    def __init__(self, vehicle, point_b, world):
        self.vehicle = vehicle
        self.point_b = point_b
        self.world = world
        self.last_update_time = self.get_simulation_time()
        self.last_vehicle_location = self.vehicle.get_location()
        self.remaining_distance = self.compute_remaining_distance()

    def get_simulation_time(self):
        """ Fetch CARLA's simulation time instead of real-world time. """
        return self.world.get_snapshot().timestamp.elapsed_seconds

    def compute_remaining_distance(self):
        """ Compute the full road-based distance using CARLA’s Global Route Planner. """
        carla_map = self.world.get_map()
        grp = GlobalRoutePlanner(carla_map, 2.0)
        route = grp.trace_route(self.vehicle.get_location(), self.point_b)
        return sum(route[i][0].transform.location.distance(route[i + 1][0].transform.location) for i in range(len(route) - 1))

    def update_remaining_distance(self):
        """ 
        Updates remaining distance every second using the full road path, 
        but uses Euclidean distance in between updates.
        """
        current_time = self.get_simulation_time()  # Use CARLA’s time
        vehicle_location = self.vehicle.get_location()

       # if current_time - self.last_update_time > 1.0:  # Update every 1 simulation second
        self.remaining_distance = self.compute_remaining_distance()
        self.last_update_time = current_time
       # else:
       #     # Estimate distance using Euclidean distance instead of speed
        #    self.remaining_distance -= vehicle_location.distance(self.last_vehicle_location)

        self.last_vehicle_location = vehicle_location  # Update last position


In [7]:
# Constants
eta_ch = 0.92  # Battery charging efficiency
P_ev_max = 80 # Maximum Power Transfer (40 kW)
Q1 = 90  # Quality factor of primary coil
Q2 = 90 # Quality factor of secondary coil
k0 = 0.2  # Nominal coupling coefficient (perfect alignment)
d_max = coil_height# Maximum misalignment before no power transfer (meters)


def get_power_transfer_efficiency(k):
    """ Computes power transfer efficiency using standard IPT equation. """
    return (k**2 * Q1 * Q2) / (1 + k**2 * Q1 * Q2)


def get_coil_center_and_orientation(corners):
    """
    Computes the center position and yaw orientation of a coil from its 4 corners.

    Args:
        corners (list of carla.Location): 4 corner points of the coil.

    Returns:
        tuple: (center_x, center_y, yaw_angle, coil_length)
    """
    # Compute the center as the average of the corners
    center_x = sum(c.x for c in corners) / 4
    center_y = sum(c.y for c in corners) / 4

    # Compute coil length from the first two corners (assuming rectangle structure)
    dx = corners[1].x - corners[0].x
    dy = corners[1].y - corners[0].y
    coil_length = math.sqrt(dx**2 + dy**2)

    # Compute yaw angle (orientation) using atan2
    yaw_angle = math.degrees(math.atan2(dy, dx))

    return carla.Location(center_x, center_y, corners[0].z), yaw_angle, coil_length




def is_on_charging_lane(vehicle, all_corners):
    """
    Checks if the vehicle is inside a charging lane and extracts coil properties.

    Args:
        vehicle (carla.Vehicle): The vehicle in CARLA.
        all_corners (list of list of carla.Location]): List of all coil corners.

    Returns:
        tuple: (True/False, coil_center, coil_orientation, coil_length)
    """
    epsilon = 0.25
    vehicle_location = vehicle.get_location()
    vehicle_x, vehicle_y = vehicle_location.x, vehicle_location.y

    for corners in all_corners:
        # Compute coil center and orientation from 4 corners
        coil_center, coil_orientation, coil_length = get_coil_center_and_orientation(corners)

        # Extract rectangle boundaries
        x_min, x_max = min(c.x for c in corners), max(c.x for c in corners)
        y_min, y_max = min(c.y for c in corners), max(c.y for c in corners)

        # Check if vehicle is inside the rectangle
        if x_min - epsilon <= vehicle_x <= x_max+epsilon and y_min -epsilon <= vehicle_y <= y_max +epsilon:
            return True, coil_center, coil_orientation, coil_length

    return False, None, None, None






def get_alignment_factor(vehicle, coil_center, coil_orientation, coil_length):
    """
    Computes alignment efficiency considering lateral, longitudinal, and angular misalignment.

    Args:
        vehicle (carla.Vehicle): The vehicle in CARLA.
        coil_center (carla.Location): Center of the charging coil.
        coil_orientation (float): Coil yaw angle in degrees.
        coil_length (float): Length of the coil in meters.

    Returns:
        tuple: (Alignment factor, Coupling coefficient k)
    """
    # Constants

    l_max = coil_length   # Max longitudinal misalignment


    # Get vehicle position
    vehicle_location = vehicle.get_location()
    vehicle_x, vehicle_y = vehicle_location.x, vehicle_location.y

    # Compute Lateral Misalignment (y-axis)
    d_y = abs(vehicle_y - coil_center.y)  # Lateral deviation
    f_lat = np.exp(- (d_y / d_max) ** 2)# if d_y <= d_max else 0

    # Compute Longitudinal Misalignment (x-axis)
    d_x = abs(vehicle_x - coil_center.x)  # Longitudinal deviation
    f_long = np.exp(- (d_x / l_max) ** 2)# if d_x <= l_max else 0

    # Compute Angular Misalignment (yaw)
    vehicle_rotation = vehicle.get_transform().rotation.yaw  # Vehicle yaw angle
    theta_misalign = abs(vehicle_rotation - coil_orientation)  # Misalignment in degrees
    theta_misalign_rad = math.radians(theta_misalign)

    f_ang = abs(np.cos(theta_misalign_rad) + 0.5 * np.sin(theta_misalign_rad))  # Ensures non-negative value

    # Compute Final Alignment Factor
    f_align = f_lat * f_long * f_ang

    # Compute Coupling Coefficient (k)
    k = k0 * f_align#f_lat * f_long  # Exponential decay model

    return f_align, k









def update_battery(vehicle, soc, total_energy, all_corners, power_model, df, delta_time, C_bat, P_aux, location, point_b, max_speed, avg_speed,battery_manager,speed_m_s):
    """
    Updates the vehicle's battery considering DWCL charging.

    Returns:
        tuple: (Updated SoC, Updated total energy, ETA)
    """
   

    # Predict power consumption
    P_net, req_P = -power_model.predict(df, verbose=0).flatten() #in kW
    P_net = P_net  * 0 if speed_m_s <=0 else P_net
    P_total = delta_time * (P_aux + P_net)  # Convert to kWh
    T_req_P = (P_aux + req_P)

    # Compute remaining distance
    #remaining_distance = abs(point_b.x - location.x)
    battery_manager.update_remaining_distance()
    remaining_distance = battery_manager.remaining_distance

    # Compute energy consumption
    energy_consumed = (1 / 3600) * P_total  # Convert W to kWh
    energy_required = (remaining_distance / max_speed) / 3600 * T_req_P
    soc_required = (energy_required / C_bat) * 100

    # Check if vehicle is inside a charging lane
    on_charging_lane, coil_center, coil_orientation, coil_length = is_on_charging_lane(vehicle, all_corners)
    if on_charging_lane:
        f_align, k = get_alignment_factor(vehicle, coil_center, coil_orientation, coil_length)
        eta_transfer = get_power_transfer_efficiency(k)
        P_ev = P_ev_max * eta_transfer * eta_ch  # Adjust charging power based on efficiency
        energy_charged = (P_ev * delta_time) / 3600  # Convert W to kWh
    else:
        energy_charged = 0  # No charging if outside DWCL
        P_ev = 0
        f_align, k,eta_transfer =0,0,0
        

    # Update battery energy and SoC
    total_energy += (energy_consumed - energy_charged)
    total_energy = max(0,total_energy)
    soc -= ((energy_consumed - energy_charged) / C_bat) * 100
    soc = min(soc, 100)  # Ensure SoC does not exceed 100%
    soc = max(soc, 0)  # Ensure SoC does not exceed 100%

    # Compute ETA (Estimated Time of Arrival)
    ETA = remaining_distance / (avg_speed + 0.01)

    return soc,soc_required, total_energy, ETA,P_ev, f_align, k,eta_transfer,remaining_distance









In [ ]:


# Pygame initialization
pygame.init()
WIDTH, HEIGHT = 1600, 900  # Window size
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("CARLA Vehicle Info Display")
font = pygame.font.Font(None, 24)
clock = pygame.time.Clock()

# Connect to CARLA
settings = world.get_settings()
settings.synchronous_mode = True
world.apply_settings(settings)

point_a = spawn_points[0]
point_b = carla.Location(
    x=start_point.location.x - length,
    y=start_point.location.y,
    z=start_point.location.z,
)

# Vehicle constants
m = 1680  # kg, Mass
g = 9.81  # m/s^2, Gravity
c_rr = 0.01  # Rolling resistance coefficient
c_d = 0.28  # Drag coefficient
A = 1.93  # m^2, Frontal area
rho = 1.20  # kg/m^3, Air density
C_bat = 24  # kWh, Battery capacity
alpha = 0.90  # Transmission efficiency
beta = 0.75  # Regeneration efficiency
P_aux = 2  # kW, Auxiliary power consumption
delta_t = 0.00001  # s, Sampling interval
max_speed = 70 / 3.6 #m/s
avg_speed = 40 / 3.6
avg_slop = 0.003
avg_acc = 1.0

battery_manager = BatteryManager(vehicle, point_b, world)

def get_slope(vehicle):
    rotation = vehicle.get_transform().rotation
    pitch = rotation.pitch
    return math.tan(math.radians(pitch))  # Slope in radians



# Initialize variables

prev_velocity = None
prev_time = None
prev_position = vehicle.get_location()  # Initialize the previous position
soc = 10  # State of Charge in percentage
total_energy = 0  # Total energy consumed in kWh
driven_distance = 0  # Total distance driven in meters
#abs(point_b.x - point_a.location.x)  # Remaining distance in meters



# Attach a camera to the vehicle
camera_bp = world.get_blueprint_library().find("sensor.camera.rgb")
camera_bp.set_attribute("image_size_x", str(WIDTH))
camera_bp.set_attribute("image_size_y", str(HEIGHT))
camera_bp.set_attribute("fov", "110")
camera_transform = carla.Transform(carla.Location(x=-1.5, z=4.0))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)
latest_image = np.zeros((HEIGHT, WIDTH, 3), dtype=np.uint8)


def process_img(image):
    global latest_image
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = np.reshape(array, (image.height, image.width, 4))
    latest_image = cv2.flip(array[:, :, :3], 1)  # Flip horizontally

camera.listen(lambda image: process_img(image))

# Agent for automatic driving
agent = BehaviorAgent(vehicle, behavior="normal")
spawn_points = world.get_map().get_spawn_points()
target_location = spawn_points[10].location
agent.set_destination(target_location)

# Load power model
power_model = load_model("c:\\Users\\HOUALEF\\Desktop\\RL_Carla\\models\\energy_model_best.hdf5")


manual_mode = True
control = carla.VehicleControl()

# Main loop
running = True
while running:
    world.tick()
    clock.tick(20)

     # Get the current position, velocity, and slope
    location = vehicle.get_location()
    velocity = vehicle.get_velocity()
    slope = get_slope(vehicle)
    start_time = world.get_snapshot().timestamp.elapsed_seconds
    
    # Calculate the speed in m/s and km/h
    speed_m_s = math.sqrt(velocity.x**2 + velocity.y**2)  # Speed in m/s
    speed_kmh = speed_m_s * 3.6  # Speed in km/h

    # Calculate acceleration
    if prev_velocity is not None and prev_time is not None:
        delta_time = world.get_snapshot().timestamp.elapsed_seconds - prev_time
        delta_velocity = speed_m_s - math.sqrt(prev_velocity.x**2 + prev_velocity.y**2)
        acceleration = delta_velocity / (delta_time +0.1) # m/s²
    else:
        acceleration = 0  # No acceleration at the start
        delta_time = 0.1


    speed_vector = np.array([speed_m_s,max_speed])
    acc_vector = np.array([acceleration,avg_acc])
    slop_vector = np.array([slope,avg_slop])
    # Calculate forces
    F_roll = c_rr * m * g  * speed_vector/ 1000 # Rolling resistance force
    F_climb = m * g * np.sin(slop_vector) * speed_vector / 1000  # Climbing force
    F_aero = 0.5 * c_d * A * rho * speed_vector**2 * speed_vector / 1000  # Aerodynamic drag force
    F_acc = m * acc_vector * speed_vector * np.array([1,0]) / 1000 # Acceleration force
    #F_traction = F_aero + F_acc + F_climb + F_roll  # Total traction force

    # Calculate power
    #if F_traction < 0:  # Regenerative braking
    #    P_net = F_traction * speed_m_s * beta
    #else:  # Driving
    #    P_net = F_traction * speed_m_s / alpha

    # Total power consumption (add auxiliary power)

    df = pd.DataFrame({
                'aeroF_P': F_aero,
                'climbF_P': F_climb,
                'rollF_P': F_roll,
                'accF_P': F_acc })
    """
    P_net,req_P = -power_model.predict(df, verbose=0).flatten()
    P_total = delta_time * (P_aux + P_net) # Convert to kW
    T_req_P = (P_aux + req_P)

    # Calculate remaining distance along the x-axis
    remaining_distance = abs(point_b.x - location.x)

    # Energy consumption over the sampling interval
    energy_consumed = (1 / 3600) * P_total  # Energy in kWh
    energy_required = (remaining_distance / max_speed)/3600 * T_req_P
    total_energy += energy_consumed

    # Update SoC
    soc -= (energy_consumed / C_bat) * 100  # Decrease SoC
    soc_required = (energy_required / C_bat) * 100
    soc=min(soc,100)
    ETA = remaining_distance / (avg_speed + 0.01)

    """

    


    soc,soc_required, total_energy, ETA,charging_power,f_align, k,eta_transfer,remaining_distance =update_battery(vehicle, soc, total_energy, all_corners, power_model, df, delta_time, C_bat, P_aux, location, point_b, max_speed, avg_speed,battery_manager,speed_m_s)

    # Calculate driven distance
    delta_distance = math.sqrt(
        (location.x - prev_position.x) ** 2 +
        (location.y - prev_position.y) ** 2
    )
    driven_distance += delta_distance

    # Manual control
    keys = pygame.key.get_pressed()
    if keys[pygame.K_m]:  # Toggle mode
        manual_mode = not manual_mode
        time.sleep(0.2)

    if manual_mode:
        control.throttle = 1.0 if keys[pygame.K_UP] else 0.0
        control.brake = 1.0 if keys[pygame.K_DOWN] else 0.0
        control.steer = -0.5 if keys[pygame.K_LEFT] else (0.5 if keys[pygame.K_RIGHT] else 0.0)
        control.reverse = keys[pygame.K_SPACE]  # Enable reverse when down arrow is pressed
    else:
        control = agent.run_step()
    vehicle.apply_control(control)

    # Pygame Display Update
    screen.fill((0, 0, 0))
    frame = cv2.cvtColor(latest_image, cv2.COLOR_BGR2RGB)
    frame = np.rot90(frame)
    frame = pygame.surfarray.make_surface(frame)
    screen.blit(frame, (0, 0))
    # Update previous values for the next iteration
    prev_velocity = velocity
    prev_position = location
    prev_time = world.get_snapshot().timestamp.elapsed_seconds

    text_data = [
        f"Driving Mode: {'Manual' if manual_mode else 'Automatic'}",
        f"Speed: {speed_kmh:.2f} km/h",
        f"SoC: {soc:.2f}%",
        f"Energy Consumed: {total_energy:.4f} kWh",
        f"ETA: {ETA:.2f}s",
        f"Required SoC: {soc_required:.2f}%",
        f"Remaining Distance: {remaining_distance:.2f}m",
        f"Charging Power: {charging_power:.2f}Kw",
        f"coupling k: {k:.2f}",
        f"alligment: {f_align:.2f}",
        f"eta_transfer: {eta_transfer:.2f}",
        f"time: {start_time:.2f}",
        
    ]
    y_offset = 30
    for text in text_data:
        screen.blit(font.render(text, True, (255, 255, 255)), (20, y_offset))
        y_offset += 30

    pygame.display.flip()

    for event in pygame.event.get():
        if event.type == pygame.QUIT or soc <= 0:
            running = False

pygame.quit()
camera.destroy()
#vehicle.destroy()
print("Simulation ended.")


In [9]:
#all imports
import carla #the sim library itself
import random #to pick random spawn point
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import time
from keras.models import load_model
import pandas as pd
import sys
sys.path.append("D:\WindowsNoEditor\PythonAPI\carla")  # Adjust this path
import carla
import pygame
from agents.navigation.global_route_planner import GlobalRoutePlanner
from agents.navigation.behavior_agent import BehaviorAgent


class BatteryManager:
    def __init__(self, vehicle, destination, world, power_model, C_bat, P_aux):
        self.vehicle = vehicle
        self.destination = destination  # Renamed from point_b
        self.world = world
        self.power_model = power_model
        self.C_bat = C_bat
        self.P_aux = P_aux
        self.last_update_time = self.get_simulation_time()
        self.last_vehicle_location = self.vehicle.get_location()
        self.remaining_distance = self.compute_remaining_distance()
        self.soc = 100  # State of Charge in percentage
        self.total_energy = 0  # Total energy consumed (kWh)
        self.prev_velocity = self.vehicle.get_velocity()  # Initialize in constructor

    
    def get_simulation_time(self):
        return self.world.get_snapshot().timestamp.elapsed_seconds

    def compute_remaining_distance(self):
        carla_map = self.world.get_map()
        grp = GlobalRoutePlanner(carla_map, 2.0)
        route = grp.trace_route(self.vehicle.get_location(), self.destination)
        return sum(route[i][0].transform.location.distance(route[i + 1][0].transform.location) for i in range(len(route) - 1))

    def update_remaining_distance(self):
        current_time = self.get_simulation_time()
        vehicle_location = self.vehicle.get_location()
        
        self.remaining_distance = self.compute_remaining_distance()
        self.last_update_time = current_time
        self.last_vehicle_location = vehicle_location

    def get_slope(self):
        rotation = self.vehicle.get_transform().rotation
        return math.tan(math.radians(rotation.pitch))

    def compute_forces(self, speed_m_s, acceleration):
        slope = self.get_slope()
        speed_vector = np.array([speed_m_s, avg_speed])
        acc_vector = np.array([acceleration, avg_acc])
        slope_vector = np.array([slope, avg_slop])
        
        F_roll = c_rr * m * g * speed_vector / 1000
        F_climb = m * g * np.sin(slope_vector) * speed_vector / 1000
        F_aero = 0.5 * c_d * A * rho * speed_vector**2 * speed_vector / 1000
        F_acc = m * acc_vector * speed_vector  / 1000
        
        return F_roll, F_climb, F_aero, F_acc

    def get_power_transfer_efficiency(self, k):
        return (k**2 * Q1 * Q2) / (1 + k**2 * Q1 * Q2)

    def get_coil_center_and_orientation(self, corners):
        center_x = sum(c.x for c in corners) / 4
        center_y = sum(c.y for c in corners) / 4
        dx = corners[1].x - corners[0].x
        dy = corners[1].y - corners[0].y
        coil_length = math.sqrt(dx**2 + dy**2)
        yaw_angle = math.degrees(math.atan2(dy, dx))
        return carla.Location(center_x, center_y, corners[0].z), yaw_angle, coil_length

    def is_on_charging_lane(self, all_corners):
        epsilon = 0.25
        vehicle_location = self.vehicle.get_location()
        vehicle_x, vehicle_y = vehicle_location.x, vehicle_location.y
        for corners in all_corners:
            coil_center, coil_orientation, coil_length = self.get_coil_center_and_orientation(corners)
            x_min, x_max = min(c.x for c in corners), max(c.x for c in corners)
            y_min, y_max = min(c.y for c in corners), max(c.y for c in corners)
            if x_min - epsilon <= vehicle_x <= x_max + epsilon and y_min - epsilon <= vehicle_y <= y_max + epsilon:
                return True, coil_center, coil_orientation, coil_length
        return False, None, None, None

    def get_alignment_factor(self, coil_center, coil_orientation, coil_length):
        l_max = coil_length
        vehicle_location = self.vehicle.get_location()
        vehicle_x, vehicle_y = vehicle_location.x, vehicle_location.y
        d_y = abs(vehicle_y - coil_center.y)
        f_lat = np.exp(- (d_y / d_max) ** 2)
        d_x = abs(vehicle_x - coil_center.x)
        f_long = np.exp(- (d_x / l_max) ** 2)
        vehicle_rotation = self.vehicle.get_transform().rotation.yaw
        theta_misalign = abs(vehicle_rotation - coil_orientation)
        theta_misalign_rad = math.radians(theta_misalign)
        f_ang = abs(np.cos(theta_misalign_rad) + 0.5 * np.sin(theta_misalign_rad))
        f_align = f_lat * f_long * f_ang
        k = k0 * f_align
        return f_align, k
    

    def get_acceleration(self,delta_time):
       
        
        if delta_time <= 0:
            return 0  # Avoid division by zero or negative time steps

        velocity = self.vehicle.get_velocity()
        speed_m_s = math.sqrt(velocity.x**2 + velocity.y**2)

        if hasattr(self, "prev_velocity") and self.prev_velocity is not None:
            prev_speed_m_s = math.sqrt(self.prev_velocity.x**2 + self.prev_velocity.y**2)
            acceleration = (speed_m_s - prev_speed_m_s) / delta_time
        else:
            acceleration = 0  # First time, assume no acceleration

        # Update for next calculation
        self.prev_velocity = velocity
        
        

        return acceleration


    


    def update_battery(self, delta_time, all_corners):
        location = self.vehicle.get_location()
        velocity = self.vehicle.get_velocity()
        speed_m_s = np.sqrt(velocity.x**2 + velocity.y**2)
        acceleration = self.get_acceleration(delta_time)
        slope = self.get_slope() * 57.2958
        F_roll, F_climb, F_aero, F_acc = self.compute_forces(speed_m_s, acceleration)
        df = pd.DataFrame({
            'aeroF_P': F_aero,
            'climbF_P': F_climb,
            'rollF_P': F_roll,
            'accF_P': F_acc
        })
        P_net, req_P = -power_model.predict(df, verbose=0).flatten() #in kW
        P_net = P_net  * 0 if speed_m_s <=0 else P_net
        P_total = delta_time * (P_aux + P_net)  # Convert to kWh
        T_req_P = (P_aux + req_P)

        # Compute remaining distance
        #remaining_distance = abs(point_b.x - location.x)
        self.update_remaining_distance()
        remaining_distance = battery_manager.remaining_distance

        # Compute energy consumption
        energy_consumed = (1 / 3600) * P_total  # Convert W to kWh
        energy_required = (remaining_distance / avg_speed) / 3600 * T_req_P
        soc_required = (energy_required / C_bat) * 100
        on_charging_lane, coil_center, coil_orientation, coil_length = self.is_on_charging_lane(all_corners)
        if on_charging_lane:
            f_align, k = self.get_alignment_factor(coil_center, coil_orientation, coil_length)
            eta_transfer = self.get_power_transfer_efficiency(k)
            P_ev = P_ev_max * eta_transfer * eta_ch
            energy_charged = (P_ev * delta_time) / 3600
        else:
            energy_charged, P_ev, f_align, k, eta_transfer = 0, 0, 0, 0, 0
        self.total_energy += (energy_consumed - energy_charged)
        self.soc -= ((energy_consumed - energy_charged) / C_bat) * 100
        self.soc = min(max(self.soc, 0), 100)
        self.total_energy = max(0,self.total_energy)
        ETA = remaining_distance / (avg_speed + 0.01)
        return self.soc, soc_required, self.total_energy, ETA, P_ev, f_align, k, eta_transfer, remaining_distance,acceleration,slope


In [16]:


# Constants
eta_ch = 0.92  # Battery charging efficiency
P_ev_max = 80  # Maximum Power Transfer (40 kW)
Q1 = 90  # Quality factor of primary coil
Q2 = 90 # Quality factor of secondary coil
k0 = 0.2  # Nominal coupling coefficient (perfect alignment)
d_max = 0.5  # Maximum lateral misalignment before no power transfer (meters)

# Vehicle constants
m = 1680  # kg, Mass
g = 9.81  # m/s^2, Gravity
c_rr = 0.01  # Rolling resistance coefficient
c_d = 0.28  # Drag coefficient
A = 1.93  # m^2, Frontal area
rho = 1.20  # kg/m^3, Air density
C_bat = 24  # kWh, Battery capacity
alpha = 0.90  # Transmission efficiency
beta = 0.75  # Regeneration efficiency
P_aux = 0  # kW, Auxiliary power consumption
max_speed = 70 / 3.6  # m/s
avg_speed = 35 / 3.6
avg_slop = 0.002
avg_acc = 0.2

# Pygame initialization
pygame.init()
WIDTH, HEIGHT = 1600, 900  # Window size
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("CARLA Vehicle Info Display")
font = pygame.font.Font(None, 24)
clock = pygame.time.Clock()

# Connect to CARLA
settings = world.get_settings()
settings.synchronous_mode = True
world.apply_settings(settings)

spawn_points = world.get_map().get_spawn_points()
destination = carla.Location(
    x=start_point.location.x - length,
    y=start_point.location.y,
    z=start_point.location.z,
)

# Load power model
power_model = load_model("c:\\Users\\HOUALEF\\Desktop\\RL_Carla\\models\\energy_model_best.hdf5")

# Initialize battery manager
battery_manager = BatteryManager(vehicle, destination, world, power_model,C_bat,P_aux)

# Attach a camera to the vehicle
camera_bp = world.get_blueprint_library().find("sensor.camera.rgb")
camera_bp.set_attribute("image_size_x", str(WIDTH))
camera_bp.set_attribute("image_size_y", str(HEIGHT))
camera_bp.set_attribute("fov", "110")
camera_transform = carla.Transform(
    carla.Location(x=-6.0, z=2.0),  # 6m behind, 2m above the vehicle
    carla.Rotation(pitch=-15)       # Optional: slight downward tilt
)
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)
latest_image = np.zeros((HEIGHT, WIDTH, 3), dtype=np.uint8)

def process_img(image):
    global latest_image
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = np.reshape(array, (image.height, image.width, 4))
    latest_image = cv2.flip(array[:, :, :3], 1)

camera.listen(lambda image: process_img(image))

# Agent for automatic driving
agent = BehaviorAgent(vehicle, behavior="normal")
agent.set_destination(destination)

manual_mode = True
control = carla.VehicleControl()

# Main loop
running = True
while running:
    world.tick()
    clock.tick(20)
    
    current_time = battery_manager.get_simulation_time()
    delta_time = current_time - battery_manager.last_update_time
    battery_manager.last_update_time = current_time
    
    soc, soc_required, total_energy, ETA, charging_power, f_align, k, eta_transfer, remaining_distance,acceleration,slope = battery_manager.update_battery(
        delta_time, all_corners
    )
    
    # Manual control
    keys = pygame.key.get_pressed()
    if keys[pygame.K_m]:
        manual_mode = not manual_mode
        time.sleep(0.2)
    
    if manual_mode:
        control.throttle = 1.0 if keys[pygame.K_UP] else 0.0
        control.brake = 1.0 if keys[pygame.K_DOWN] else 0.0
        control.steer = -0.5 if keys[pygame.K_LEFT] else (0.5 if keys[pygame.K_RIGHT] else 0.0)
        control.reverse = keys[pygame.K_SPACE]
    else:
        control = agent.run_step()
    
    vehicle.apply_control(control)
    
    # Pygame Display Update
    screen.fill((0, 0, 0))
    frame = cv2.cvtColor(latest_image, cv2.COLOR_BGR2RGB)
    frame = np.rot90(frame)
    frame = pygame.surfarray.make_surface(frame)
    screen.blit(frame, (0, 0))
    
    text_data = [
        f"Driving Mode: {'Manual' if manual_mode else 'Automatic'}",
        f"Speed: {vehicle.get_velocity().length() * 3.6:.2f} km/h",
        f"SoC: {soc:.2f}%",
        f"Energy Consumed: {total_energy:.4f} kWh",
        f"ETA: {ETA:.2f}s",
        f"Required SoC: {soc_required:.2f}%",
        f"Remaining Distance: {remaining_distance:.2f}m",
        f"Charging Power: {charging_power:.2f} kW",
        f"Coupling k: {k:.2f}",
        f"Alignment: {f_align:.2f}",
        f"Eta Transfer: {eta_transfer:.2f}",
        f"Delta Time: {delta_time:.2f} s" ,
        f"Acceleration: {acceleration:.2f} m/s²",
        f"Slope: {slope:.2f} °",
    ]
    
    y_offset = 30
    for text in text_data:
        screen.blit(font.render(text, True, (255, 255, 255)), (20, y_offset))
        y_offset += 30
    
    pygame.display.flip()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT or soc <= 0:
            running = False

pygame.quit()
camera.destroy()
print("Simulation ended.")


Simulation ended.
